# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lihue,21.9789,-159.3672,26.77,72,0,6.69,US,1720241177
1,1,olonkinbyen,70.9221,-8.7187,1.40,93,100,5.02,SJ,1720241249
2,2,bredasdorp,-34.5322,20.0403,8.01,77,100,2.22,ZA,1720241250
3,3,puerto natales,-51.7236,-72.4875,-6.51,90,36,1.39,CL,1720241251
4,4,ushuaia,-54.8000,-68.3000,-2.19,86,75,2.57,AR,1720241252


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
mycity_data_df = city_data_df[(city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 21) & (city_data_df['Wind Speed'] < 4.5) 
                    & (city_data_df['Cloudiness'] == 0)] 

# Drop any rows with null values
mycity_data_df = mycity_data_df.dropna()

# Display sample data
mycity_data_df.head(40)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
41,41,remire-montjoly,4.9167,-52.2667,24.02,100,0,1.03,GF,1720241298
61,61,murzuq,25.9155,13.9184,26.81,31,0,3.74,LY,1720241322
111,111,danville,36.5860,-79.3950,24.32,94,0,1.54,US,1720241388
135,135,los altos,37.3852,-122.1141,24.04,66,0,3.60,US,1720241361
143,143,centerville,39.6284,-84.1594,21.62,94,0,2.80,US,1720241426
162,162,luleburgaz,41.4039,27.3592,25.19,43,0,2.35,TR,1720241452
199,199,matoury,4.8483,-52.3317,23.98,100,0,1.03,GF,1720241498
315,315,hamilton,39.1834,-84.5333,22.07,94,0,2.06,US,1720241640
323,323,mckinleyville,40.9465,-124.1006,22.92,83,0,0.00,US,1720241649
340,340,roma,43.2128,-75.4557,24.85,90,0,1.54,US,1720241379


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = mycity_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(20)

,City,Country,Lat,Lng,Humidity,Hotel Name
41,remire-montjoly,GF,4.9167,-52.2667,100,
61,murzuq,LY,25.9155,13.9184,31,
111,danville,US,36.5860,-79.3950,94,
135,los altos,US,37.3852,-122.1141,66,
143,centerville,US,39.6284,-84.1594,94,
162,luleburgaz,TR,41.4039,27.3592,43,
199,matoury,GF,4.8483,-52.3317,100,
315,hamilton,US,39.1834,-84.5333,94,
323,mckinleyville,US,40.9465,-124.1006,83,
340,roma,US,43.2128,-75.4557,90,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
limit = 20
categories = "accommodation.hotel"

params = {
    "categories":categories,
    "apiKey":geoapify_key,
    "limit":limit
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
remire-montjoly - nearest hotel: Complexe Belova
murzuq - nearest hotel: فندق باريس - مرزق
danville - nearest hotel: Holiday Inn Express Danville
los altos - nearest hotel: Mountain View Inn
centerville - nearest hotel: Holiday Inn Express & Suites
luleburgaz - nearest hotel: Bulgaz Izer
matoury - nearest hotel: La Marmotte
hamilton - nearest hotel: North Vista Manor
mckinleyville - nearest hotel: Ramada Arcata
roma - nearest hotel: Wingate by Wyndham Rome
broome - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
argostolion - nearest hotel: Ainos Hotel
lensk - nearest hotel: No hotel found
prince george - nearest hotel: Travelodge by Wyndham Prince George
florida - nearest hotel: Chassahowitzka Hotel
guercif - nearest hotel: No hotel found
siwa oasis - nearest hotel: Flower of the city hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
41,remire-montjoly,GF,4.9167,-52.2667,100,Complexe Belova
61,murzuq,LY,25.9155,13.9184,31,فندق باريس - مرزق
111,danville,US,36.5860,-79.3950,94,Holiday Inn Express Danville
135,los altos,US,37.3852,-122.1141,66,Mountain View Inn
143,centerville,US,39.6284,-84.1594,94,Holiday Inn Express & Suites
162,luleburgaz,TR,41.4039,27.3592,43,Bulgaz Izer
199,matoury,GF,4.8483,-52.3317,100,La Marmotte
315,hamilton,US,39.1834,-84.5333,94,North Vista Manor
323,mckinleyville,US,40.9465,-124.1006,83,Ramada Arcata
340,roma,US,43.2128,-75.4557,90,Wingate by Wyndham Rome


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Country", "Hotel Name"]
)

# Display the map
map_plot_2


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)